In [1]:
from ctypes import *
import time
from sys import path
from os import sep

dwf = cdll.LoadLibrary("libdwf.so") #so=Source Object
constants_path = "/usr/share/digilent/wavefoms/samples/py"
path.append(constants_path)
import dwfconstants as constants

In [2]:
dwf.FDwfDeviceCloseAll()
filter_flags = c_int(constants.enumfilterType.value | constants.enumfilterUSB.value)
device_count = c_int()
dwf.FDwfEnum(filter_flags, byref(device_count))

hdwf = c_int()
dwf.FDwfDeviceOpen(c_int(-1), byref(hdwf))
if hdwf.value == 0:
    raise RuntimeError("Failed to open device.")
print(hdwf.value)

dwf.FDwfAnalogIOReset(hdwf)
dwf.FDwfAnalogIOChannelNodeSet(hdwf, c_int(0), c_int(1), c_double(5.0))
dwf.FDwfAnalogIOChannelNodeSet(hdwf, c_int(0), c_int(0), c_double(1))
dwf.FDwfAnalogIOEnableSet(hdwf, c_int(1))
time.sleep(0.5)

vpos = c_double()
dwf.FDwfAnalogIOStatus(hdwf)
dwf.FDwfAnalogIOChannelNodeStatus(hdwf, c_int(0), c_int(1), byref(vpos))
print(f"Power: +{vpos.value:.2f}V")

1
Power: +5.00V


In [3]:
DEVICE_ADDR = 0x52
ENABLE_REG = 0x80
ATIME_REG = 0x81
CONTROL_REG = 0x8F
CDATAL_REG = 0x94
CDATAH_REG = 0x95
RDATAL_REG = 0x96
RDATAH_REG = 0x97
GDATAL_REG = 0x98
GDATAH_REG = 0x99
BDATAL_REG = 0x9A
BDATAH_REG = 0x9B

def write_register(reg, value):
    iNak = c_int()
    rgTX = (c_ubyte * 2)(reg, value)
    dwf.FDwfDigitalI2cWrite(hdwf, c_int(DEVICE_ADDR), rgTX, c_int(2), byref(iNak))
    return iNak.value == 0

def read_register(reg):
    iNak = c_int()
    rgTX = (c_ubyte * 1)(reg)
    dwf.FDwfDigitalI2cWrite(hdwf, c_int(DEVICE_ADDR), rgTX, c_int(1), byref(iNak))
    if iNak.value == 0:
        rgRX = (c_ubyte * 1)()
        dwf.FDwfDigitalI2cRead(hdwf, c_int(DEVICE_ADDR), rgRX, c_int(1), byref(iNak))
        if iNak.value == 0:
            return rgRX[0]
    return None

dwf.FDwfDigitalI2cRateSet(hdwf, c_double(100e3))
dwf.FDwfDigitalI2cSclSet(hdwf, c_int(0))
dwf.FDwfDigitalI2cSdaSet(hdwf, c_int(1))
iNak = c_int()
dwf.FDwfDigitalI2cClear(hdwf, byref(iNak))

1

In [4]:
write_register(ENABLE_REG, 0x01)
time.sleep(0.01)
write_register(ENABLE_REG, 0x03)
write_register(ATIME_REG, 0xF6)
write_register(CONTROL_REG, 0x00)

def read_colors():
    time.sleep(0.1)
    
    clear_low = read_register(CDATAL_REG)
    clear_high = read_register(CDATAH_REG)
    red_low = read_register(RDATAL_REG)
    red_high = read_register(RDATAH_REG)
    green_low = read_register(GDATAL_REG)
    green_high = read_register(GDATAH_REG)
    blue_low = read_register(BDATAL_REG)
    blue_high = read_register(BDATAH_REG)
    
    if all(val is not None for val in [clear_low, clear_high, red_low, red_high, green_low, green_high, blue_low, blue_high]):
        clear = (clear_high << 8) | clear_low
        red = (red_high << 8) | red_low
        green = (green_high << 8) | green_low
        blue = (blue_high << 8) | blue_low
        return {'clear': clear, 'red': red, 'green': green, 'blue': blue}
    return None

In [7]:
def cleanup():
    dwf.FDwfDeviceClose(hdwf)

In [8]:
cleanup()
#always call this function after your work is done

In [5]:

write_register(CONTROL_REG,0)
for i in range(256):
    write_register(ATIME_REG,255-i)
    colors = read_colors()

    print(f"Clear: {colors['clear']}, R: {colors['red']}, G: {colors['green']}, B: {colors['blue']}")

Clear: 9, R: 1, G: 3, B: 1
Clear: 10, R: 4, G: 6, B: 5
Clear: 22, R: 6, G: 8, B: 7
Clear: 28, R: 7, G: 10, B: 9
Clear: 32, R: 9, G: 13, B: 11
Clear: 39, R: 10, G: 14, B: 12
Clear: 49, R: 13, G: 17, B: 15
Clear: 55, R: 14, G: 20, B: 17
Clear: 63, R: 16, G: 23, B: 20
Clear: 67, R: 18, G: 26, B: 23
Clear: 73, R: 19, G: 26, B: 23
Clear: 82, R: 21, G: 30, B: 26
Clear: 89, R: 23, G: 32, B: 28
Clear: 93, R: 24, G: 34, B: 29
Clear: 104, R: 27, G: 37, B: 33
Clear: 111, R: 28, G: 40, B: 35
Clear: 118, R: 30, G: 41, B: 36
Clear: 122, R: 31, G: 44, B: 38
Clear: 128, R: 33, G: 46, B: 40
Clear: 137, R: 35, G: 49, B: 43
Clear: 143, R: 37, G: 52, B: 45
Clear: 149, R: 38, G: 54, B: 47
Clear: 161, R: 41, G: 58, B: 51
Clear: 166, R: 43, G: 60, B: 52
Clear: 173, R: 44, G: 62, B: 54
Clear: 171, R: 46, G: 64, B: 57
Clear: 176, R: 45, G: 63, B: 59
Clear: 189, R: 48, G: 68, B: 59
Clear: 199, R: 51, G: 72, B: 63
Clear: 200, R: 51, G: 72, B: 64
Clear: 206, R: 53, G: 74, B: 65
Clear: 216, R: 57, G: 80, B: 70
Cle

Device closed
